In [1]:
import json
import pandas as pd

# Load JSON file
with open("C:/Users/camil/IMAG/DeepDRA/metadata.cohort.2025-06-12.json", "r", encoding="utf-8") as file:
    data = json.load(file)

print(json.dumps(data[0], indent=2))

{
  "data_format": "BCR XML",
  "access": "open",
  "associated_entities": [
    {
      "entity_submitter_id": "TCGA-S3-A6ZH",
      "entity_type": "case",
      "case_id": "ec461eaf-f5cb-4225-a9a0-59d8e6c17ea6",
      "entity_id": "ec461eaf-f5cb-4225-a9a0-59d8e6c17ea6"
    }
  ],
  "file_name": "nationwidechildrens.org_biospecimen.TCGA-S3-A6ZH.xml",
  "md5sum": "d59fd59d5038db6fe3eff5cdbb18e61f",
  "file_id": "44612b57-514a-4681-a7d3-e32dd07cf1f5",
  "data_type": "Biospecimen Supplement",
  "submitter_id": "nationwidechildrens.org_biospecimen.TCGA-S3-A6ZH.xml",
  "data_category": "Biospecimen",
  "state": "released",
  "file_size": 53964
}


In [2]:
# Extraire les paires 'entity_submitter_id' et 'case_id'
rows = []

for entry in data:
    entities = entry.get("associated_entities", [])
    for entity in entities:
        submitter_id = entity.get("entity_submitter_id")
        case_id = entity.get("case_id")
        if submitter_id and case_id:
            rows.append({"entity_submitter_id": submitter_id, "case_id": case_id})

# Créer un DataFrame
df = pd.DataFrame(rows)

# Vérification
print(f"Nombre d'entrées extraites : {len(df)}")

Nombre d'entrées extraites : 242418


In [3]:
# supprimer les doublons sur 'entity_submitter_id' (garder la première occurrence)
df = df.drop_duplicates(subset='entity_submitter_id', keep='first')

# supprimer les doublons sur 'case_id' dans le DataFrame restant
df = df.drop_duplicates(subset='case_id', keep='first')

# Vérification
print(f"Nombre d'entrées extraites : {len(df)}")

Nombre d'entrées extraites : 10013


In [4]:
# Only keep the 12 first characters of the entity_submitter_id
df["entity_submitter_id"] = df["entity_submitter_id"].str[:12]

# Vérification
print(f"Nombre d'entrées extraites : {len(df)}")

Nombre d'entrées extraites : 10013


In [5]:
# Charger le fichier CSV contenant les TCGA_ID
tcga_ids = pd.read_csv("TCGA_IDs.csv")

# Trouver les IDs communs
common_ids = set(tcga_ids["TCGA_ID"]) & set(df["entity_submitter_id"])

# Compter combien d’IDs sont communs
print(f"Nombre d'IDs communs : {len(common_ids)}")


Nombre d'IDs communs : 714


In [6]:
df = df[df["entity_submitter_id"].isin(common_ids)]
print(f"df filtré : {df.shape[0]} lignes")  # devrait afficher 714

# Sauvegarder au format TSV
df.to_csv("C:/Users/camil/IMAG/DeepDRA/tcga_ids_mapping.tsv", sep="\t", index=False)

df filtré : 714 lignes
